In [10]:
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import feature_selection
import seaborn as sns
import openpyxl

In [11]:
janela = 'diaria'
book_name='bookx'
df = pd.read_csv("P:\\ciencia_de_dados\\Correlacao_de_fundos\\Bases\\"  + book_name + "_" + janela + ".csv",delimiter=',',decimal='.',parse_dates=True)
df = df.dropna().drop_duplicates().rename(columns={'Unnamed: 0':'data'})
df.head()

,data,JPEIGLSP Index,BRL Curncy,CAD Curncy,AUD Curncy,DXY Index
2,2021-10-20,322.75,5.5684,1.2363,0.7474,93.734
3,2021-10-19,322.75,5.5847,1.2379,0.7411,93.953
4,2021-10-18,328.24,5.5127,1.2368,0.7421,93.937
5,2021-10-15,325.05,5.4608,1.237,0.7416,93.956
6,2021-10-14,330.46,5.5126,1.2443,0.7379,94.08


In [12]:
df = df.melt(id_vars='data', value_vars=list(df.columns[1:]), value_name="FinancialPrice",var_name='Product')

In [13]:
#Calculo das colunas temporais
df['data']=pd.to_datetime(df['data'],format = '%Y-%m-%d')
df['MesAno'] = df['data'].dt.strftime('%m-%Y')
df['Mes_nomial']=df['data'].dt.strftime('%b')
df['Mes']=df['data'].dt.strftime('%m').astype(int)
df['Ano']=df['data'].dt.strftime('%Y').astype(int)

In [14]:
if janela == 'diaria':
    df = df.groupby('MesAno').agg(['max'])
    df.columns = df.columns.droplevel(1)
    df = df.reset_index()

In [15]:
df.sort_values(by=["Product",'data'],ascending=False,inplace=True)

In [16]:
df

,MesAno,data,Product,FinancialPrice,Mes_nomial,Mes,Ano
117,10-2021,2021-10-20,JPEIGLSP Index,94.516,Oct,10,2021
105,09-2021,2021-09-30,JPEIGLSP Index,94.338,Sep,9,2021
93,08-2021,2021-08-31,JPEIGLSP Index,93.568,Aug,8,2021
81,07-2021,2021-07-30,JPEIGLSP Index,92.973,Jul,7,2021
69,06-2021,2021-06-30,JPEIGLSP Index,92.225,Jun,6,2021
...,...,...,...,...,...,...,...
70,07-2010,2010-07-30,JPEIGLSP Index,86.019,Jul,7,2010
58,06-2010,2010-06-30,JPEIGLSP Index,88.405,Jun,6,2010
46,05-2010,2010-05-31,JPEIGLSP Index,87.162,May,5,2010
34,04-2010,2010-04-30,JPEIGLSP Index,82.381,Apr,4,2010


In [17]:
#Cria a coluna retorno e seta NaN na primeira linha de cada fundo
df['FinancialPrice']=pd.to_numeric(df["FinancialPrice"], downcast="float")
df["Retorno_1"] = (df.FinancialPrice.shift())/(df.FinancialPrice)
df.loc[df.drop_duplicates('Product').index.values,'Retorno_1'] = np.NaN
df['Retorno'] = df.Retorno_1.shift(periods=-1)
df['Retorno'] = (df["Retorno_1"]-1)

In [18]:
df.to_csv('P:\\ciencia_de_dados\\Correlacao_de_fundos\\Desenvolvimento\\1DataPrep\\base_'+ book_name + '.csv',sep =';',decimal=',')